In [ ]:
import tensorflow as tf
import cv2 
import numpy as np
import glob
from scipy.io import loadmat

landmark_path = r"E:\AI\FaceDataset\300W_LP\landmarks"
landmark_path_folders = glob.glob(landmark_path + r"\*")
landmark_anno_list = []

for f in landmark_path_folders:
    landmark_anno_list += glob.glob(f + r"\*.mat")
#print(landmark_anno_list.__len__()) 61225
#landmark_anno_list 例：E:\AI\FaceDataset\300W_LP\landmarks\AFW\AFW_9545523490_1_3_pts.mat
#             图片地址：E:\AI\FaceDataset\300W_LP\AFW\AFW_9545523490_1_3.jpg
#格式对应

writer_train = tf.python_io.TFRecordWriter("train.tfrecords")
writer_test = tf.python_io.TFRecordWriter("test.tfrecords")

for idx in range(landmark_anno_list.__len__()):
    landmark_info = landmark_anno_list[idx]
    im_path = landmark_info.replace("300W_LP\landmarks","300W_LP").replace("_pts.mat",".jpg")
    landmark = loadmat(landmark_info)['pts_2d']
    #对数据截图
    x_max = int(np.max(landmark[0:68,0]))
    x_min = int(np.min(landmark[0:68,0]))
    y_max = int(np.max(landmark[0:68,1]))
    y_min = int(np.min(landmark[0:68,1]))
    #扩大bbox范围
    y_min = int(y_min - (y_max-y_min)*0.3)
    y_max = int(y_max + (y_max-y_min)*0.1)
    x_min = int(x_min - (x_max-x_min)*0.05)
    x_max = int(x_max + (x_max-x_min)*0.05)
    
    """
    im_data = cv2.imread(im_path)
    cv2.rectangle(im_data,(x_min,y_min),(x_max,y_max),(0,255,255),2)
    for i in range(68):
        cv2.circle(im_data,(int(landmark[i][0]),int(landmark[i][1])),1,(64,250,125),2)#数据，坐标，圆半径，颜色，线条粗细
    cv2.imshow('img',im_data)
    cv2.waitKey(0)
    """
    im_data = cv2.imread(im_path)
    face_data = im_data[y_min:y_max,x_min:x_max]
    shape = face_data.shape
    im_point = []

    for p in range(68):
        im_point.append((landmark[p][0] - x_min) * 1.0 / shape[1])
        im_point.append((landmark[p][1] - y_min) * 1.0 / shape[0]) #每个点的归一化坐标
    face_data = cv2.resize(face_data,(128,128))
    ex = tf.train.Example(
        features = tf.train.Features(
            feature = {
                "image" : tf.train.Feature(
                    bytes_list = tf.train.BytesList(value = [face_data.tobytes()])
                ),
                "label" : tf.train.Feature(
                    float_list = tf.train.FloatList(value = im_point)
                )
            }
        )
    )
    if idx > landmark_anno_list.__len__() * 0.9:
        writer_test.write(ex.SerializeToString())
    else:
        writer_train.write(ex.SerializeToString())
        
writer_test.close()
writer_train.close()


E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
